In [1]:
import gymnasium as gym
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from collections import deque
import random
from pong_env import PongEnv  # Import our custom Pong environment
from datetime import datetime
import os

2025-03-06 22:25:20.619202: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741321520.637826  248777 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741321520.643485  248777 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-06 22:25:20.664028: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:

print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
# Create the environment
env = PongEnv(render_mode=False)

# Build the DQL model
model = Sequential([
    Dense(24, activation="relu", input_shape=(5,)),
    Dense(24, activation="relu"),
    Dense(env.action_space.n, activation="linear")
])
model.compile(loss="mse", optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))



/home/oasc/projvenv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1741321529.938024  248777 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6704 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1070 with Max-Q Design, pci bus id: 0000:01:00.0, compute capability: 6.1


In [26]:
model = tf.keras.models.load_model("my_model.keras")
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 24)             │           144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 24)             │           600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 3)              │            75 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,459 (9.61 KB)

 Trainable params: 819 (3.20 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,640 (6.41 KB)

In [55]:
state = env.reset()
state = tf.reshape(state, [1, 5])
done = False
total_reward = 0

In [58]:
model.predict(state, verbose=0)

array([[-149.20337, -153.59764, -142.40474]], dtype=float32)

In [53]:
memory = deque(maxlen=2000)
gamma = 0.95  # Discount factor
epsilon = .01  # Exploration rate
epsilon_min = 0.01
epsilon_decay = 0.995
mini_batch_size = 32
rewards_per_episode = []
epsilon_history = []

In [57]:
while not done:
    if np.random.rand() <= epsilon:
        action = env.action_space.sample()  # Explore , go up down or stay
    else:
        action = np.argmax(model.predict(state, verbose=0))  # Exploit 

    next_state, reward, done, _ = env.step(action)
    next_state = np.reshape(next_state, [1, 5])
    memory.append((state, action, reward, next_state, done))

    state = next_state
    total_reward += reward
    rewards_per_episode.append(rewards_per_episode)
    epsilon_history.append(epsilon)

In [86]:
start_time = datetime.now()
state = env.reset()
time_reset = start_time-datetime.now()
# print(f'Reset time: {(start_time-datetime.now()).total_seconds()}')
state = np.reshape(state, [1, 5])
done = False
total_reward = 0
start_time = datetime.now()
while not done:
    if np.random.rand() <= epsilon:
        action = env.action_space.sample()  # Explore , go up down or stay
    else:
        action = np.argmax(model.predict(state, verbose=0))  # Exploit 

    next_state, reward, done, _ = env.step(action)
    next_state = np.reshape(next_state, [1, 5])
    memory.append((state, action, reward, next_state, done))

    state = next_state
    total_reward += reward
    rewards_per_episode.append(rewards_per_episode)
    epsilon_history.append(epsilon)
time_episode = datetime.now()-start_time

print(time_episode.total_seconds())

0.007499


In [87]:
print(memory[0][1])

2


In [97]:
batch = random.sample(memory, 32)

In [114]:
batch_states = tf.convert_to_tensor([exp[0] for exp in batch], dtype=tf.float32)
batch_next_states = tf.convert_to_tensor([exp[3] for exp in batch], dtype=tf.float32)
# reshape 
batch_states = tf.squeeze(batch_states,1)
batch_next_states =  tf.squeeze(batch_next_states,1)

print(tf.shape(batch_states))
print(tf.shape(batch_next_states))


tf.Tensor([32  5], shape=(2,), dtype=int32)
tf.Tensor([32  5], shape=(2,), dtype=int32)


In [115]:
# Batch predictions (happens on GPU if available)
batch_q_values = model(batch_states, training=False)  # Use model directly
batch_next_q_values = model(batch_next_states, training=False)

In [110]:
# Reduce exploration
if epsilon > epsilon_min:
    epsilon *= epsilon_decay

start_time = datetime.now()
# Train with random samples
# if len(memory) > 32:
#     batch = random.sample(memory, 32)
#     for state, action, reward, next_state, done in batch:
#         target = reward
#         if not done:
#             target += gamma * np.amax(model.predict(next_state, verbose=0))
#         target_f = model.predict(state, verbose=0)
#         target_f[0][action] = target
#         model.fit(state, target_f, epochs=1, verbose=0)


# Train with random samples
if len(memory) > 32:
    batch = random.sample(memory, 32)

    # Convert to TensorFlow tensors for better GPU utilization
    batch_states = tf.convert_to_tensor([exp[0] for exp in batch], dtype=tf.float32)
    batch_next_states = tf.convert_to_tensor([exp[3] for exp in batch], dtype=tf.float32)

    # Batch predictions (happens on GPU if available)
    batch_q_values = model(batch_states, training=False)  # Use model directly
    batch_next_q_values = model(batch_next_states, training=False)

    # Compute targets
    targets = []
    for i, (state, action, reward, next_state, done) in enumerate(batch):
        target = reward
        if not done:
            target += gamma * tf.reduce_max(batch_next_q_values[i])  # Tensor operation
        target_f = batch_q_values[i].numpy()  # Convert to NumPy to update the action
        target_f[action] = target
        targets.append(target_f)

    # Convert to tensor
    targets = tf.convert_to_tensor(targets, dtype=tf.float32)

    # Train once on the entire batch
    model.fit(batch_states, targets, epochs=1, verbose=0)

ValueError: Exception encountered when calling Sequential.call().

[1mInvalid input shape for input [[[233.32195   144.24315   102.76389    -2.3125901  -1.9110014]]

 [[243.32195    63.302483   35.87884    -2.3125901  -1.9110014]]

 [[233.32195    44.80176    20.590832   -2.3125901  -1.9110014]]

 [[238.32195   167.36905   121.87391    -2.3125901  -1.9110014]]

 [[223.32195   139.61797    98.94189    -2.3125901  -1.9110014]]

 [[228.32195    33.23881    11.035825   -2.3125901  -1.9110014]]

 [[233.32195   158.11868   114.229904   -2.3125901  -1.9110014]]

 [[243.32195   202.05789   150.53893    -2.3125901  -1.9110014]]

 [[213.32195   111.86688    76.00987    -2.3125901  -1.9110014]]

 [[253.32195   315.37482   244.178      -2.3125901  -1.9110014]]

 [[238.32195    70.24026    41.611847   -2.3125901  -1.9110014]]

 [[243.32195   208.99567   156.27193    -2.3125901  -1.9110014]]

 [[228.32195    47.114353   22.501833   -2.3125901  -1.9110014]]

 [[238.32195   151.18091   108.496895   -

In [1]:
for event in pygame.event.get():
    if event.type == pygame.QUIT:
        run = False

NameError: name 'pygame' is not defined